In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert")

Device set to use cuda:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/FinBert/combined_clean_news.csv"
df = pd.read_csv(path)
df = df[0:700866]

In [ ]:
title = df['title'].to_list()
print(len(title))

700866


In [ ]:
import numpy as np

results = []

batch_size = 1000
for i in range(0, len(title), batch_size):
    prediction = pipe(title[i: i + batch_size])
    results.extend([d['label'] for d in prediction])

    if (i + batch_size) % 10000 == 0:
        print(len(results))
        np.save('/content/drive/MyDrive/FinBert/sentiment1.npy', np.array(results))

    print(f"Proccessed: {i + batch_size}")

Proccessed: 1000
Proccessed: 2000
Proccessed: 3000
Proccessed: 4000
Proccessed: 5000
Proccessed: 6000
Proccessed: 7000
Proccessed: 8000
Proccessed: 9000


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


10000
Proccessed: 10000
Proccessed: 11000
Proccessed: 12000
Proccessed: 13000
Proccessed: 14000
Proccessed: 15000
Proccessed: 16000
Proccessed: 17000
Proccessed: 18000
Proccessed: 19000
20000
Proccessed: 20000
Proccessed: 21000
Proccessed: 22000
Proccessed: 23000
Proccessed: 24000
Proccessed: 25000
Proccessed: 26000
Proccessed: 27000
Proccessed: 28000
Proccessed: 29000
30000
Proccessed: 30000
Proccessed: 31000
Proccessed: 32000
Proccessed: 33000
Proccessed: 34000
Proccessed: 35000
Proccessed: 36000
Proccessed: 37000
Proccessed: 38000
Proccessed: 39000
40000
Proccessed: 40000
Proccessed: 41000
Proccessed: 42000
Proccessed: 43000
Proccessed: 44000
Proccessed: 45000
Proccessed: 46000
Proccessed: 47000
Proccessed: 48000
Proccessed: 49000
50000
Proccessed: 50000
Proccessed: 51000
Proccessed: 52000
Proccessed: 53000
Proccessed: 54000
Proccessed: 55000
Proccessed: 56000
Proccessed: 57000
Proccessed: 58000
Proccessed: 59000
60000
Proccessed: 60000
Proccessed: 61000
Proccessed: 62000
Proccessed

In [ ]:
np.save('/content/drive/MyDrive/FinBert/sentiment1-1.npy', np.array(results))

In [ ]:
sentiment1 = np.load("/content/drive/MyDrive/FinBert/sentiment1-1.npy")
print(len(sentiment1))
sentiment2 = np.load("/content/drive/MyDrive/FinBert/sentiment2-2.npy")
print(len(sentiment2))
all_sentiment = np.concatenate((sentiment1, sentiment2), axis=0)
print(len(all_sentiment))

700866
700866
1401732


In [ ]:
path = "/content/drive/MyDrive/FinBert/combined_clean_news.csv"
df = pd.read_csv(path)
df['sentiment'] = all_sentiment

         date symbol                                              title  \
0  2023-12-16      A  Interesting A Put And Call Options For August ...   
1  2023-12-12      A  Wolfe Research Initiates Coverage of Agilent T...   
2  2023-12-12      A  Agilent Technologies Reaches Analyst Target Price   
3  2023-12-07      A  Agilent (A) Enhances BioTek Cytation C10 With ...   
4  2023-12-07      A  Pre-Market Most Active for Dec 7, 2023 : SQQQ,...   
5  2023-12-05      A                      A Quantitative Stock Analysis   
6  2023-11-24      A  The Zacks Analyst Blog Highlights Visa, Marrio...   
7  2023-11-22      A  3 Biotech Stocks With Big-Time Breakthrough Po...   
8  2023-11-22      A  Top Research Reports for Visa, Marriott Intern...   
9  2023-11-21      A  Tuesday Sector Leaders: Healthcare, Consumer P...   

  sentiment  
0   neutral  
1  positive  
2  positive  
3  positive  
4   neutral  
5   neutral  
6   neutral  
7  positive  
8   neutral  
9   neutral  


In [ ]:
df.head(10)
print(len(df))

1401732


In [ ]:
df.to_csv("/content/drive/MyDrive/FinBert/processed.csv", index=False)w

In [ ]:
df_all = pd.read_csv("/content/drive/MyDrive/FinBert/processed.csv")

In [ ]:
test = df_all.iloc[998]
print(test)
print(test['title'])
prediction = pipe(test['title'])
print(prediction)

date                                                2018-03-27
symbol                                                       A
title        Agilent Technologies Shows Rising Relative Str...
sentiment                                             positive
Name: 998, dtype: object
Agilent Technologies Shows Rising Relative Strength; Still Shy Of Key Benchmark
[{'label': 'positive', 'score': 0.9505119323730469}]


In [ ]:
df_all['sentiment'].value_counts()

,count
sentiment,
neutral,871086
positive,314153
negative,216493
